In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os

# Define parameters
img_width, img_height = 256, 256
batch_size = 32
epochs = 50
split_dir = 'Split_Data'

# Define data directories
train_data_dir = os.path.join(split_dir, 'train')
validation_data_dir = os.path.join(split_dir, 'val')
test_data_dir = os.path.join(split_dir, 'test')

# Create ImageDataGenerators for train, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Create a CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Adjust this line to the number of classes
])

model.summary()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Save the model
model.save('vehicle_classification_cnn_model.h5')

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Calculate and print F1 Score, Precision, and Recall
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Get true labels and predictions
true_labels = test_generator.classes
predictions = model.predict(test_generator, steps=test_generator.samples // batch_size)
predicted_labels = np.argmax(predictions, axis=1)

# Print classification report
print(classification_report(true_labels, predicted_labels, target_names=list(test_generator.class_indices.keys())))


2024-05-27 18:25:27.445436: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 18:25:27.472464: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 18:25:27.473024: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 18:25:28.075164: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 480 images belonging to 8 classes.
Found 160 images belonging to 8 classes.
Found 160 images belonging to 8 classes.


2024-05-27 18:25:29.038554: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-27 18:25:29.038885: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/50
15/15 [==============================] - 37s 2s/step - loss: 5.0477 - accuracy: 0.2729 - val_loss: 1.3421 - val_accuracy: 0.5375
Epoch 2/50
15/15 [==============================] - 36s 2s/step - loss: 0.9308 - accuracy: 0.6792 - val_loss: 1.0044 - val_accuracy: 0.6250
Epoch 3/50
15/15 [==============================] - 36s 2s/step - loss: 0.5060 - accuracy: 0.8250 - val_loss: 0.9267 - val_accuracy: 0.6625
Epoch 4/50
15/15 [==============================] - 36s 2s/step - loss: 0.3300 - accuracy: 0.8729 - val_loss: 0.8325 - val_accuracy: 0.7188
Epoch 5/50
15/15 [==============================] - 36s 2s/step - loss: 0.2554 - accuracy: 0.9208 - val_loss: 0.9998 - val_accuracy: 0.6687
Epoch 6/50
15/15 [==============================] - 36s 2s/step - loss: 0.1852 - accuracy: 0.9375 - val_loss: 0.8258 - val_accuracy: 0.7250
Epoch 7/50
15/15 [==============================] - 36s 2s/step - loss: 0.1069 - accuracy: 0.9708 - val_loss: 0.8775 - val_accuracy: 0.7125
Epoch 8/50
15/15 [==

/opt/conda/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5/5 [==============================] - 9s 2s/step - loss: 0.7659 - accuracy: 0.7750
Test Accuracy: 77.50%
5/5 [==============================] - 9s 2s/step
Confusion Matrix
[[18  0  0  0  0  1  1  0]
 [ 0 16  3  0  0  1  0  0]
 [ 0  0 16  0  1  0  3  0]
 [ 0  0  0 17  0  2  1  0]
 [ 0  1  3  1 10  2  2  1]
 [ 0  0  0  1  0 18  1  0]
 [ 1  0  0  0  0  1 18  0]
 [ 1  0  5  0  2  0  1 11]]
Classification Report
              precision    recall  f1-score   support

         bus       0.90      0.90      0.90        20
         car       0.94      0.80      0.86        20
     minivan       0.59      0.80      0.68        20
   motorbike       0.89      0.85      0.87        20
      pickup       0.77      0.50      0.61        20
     scooter       0.72      0.90      0.80        20
       truck       0.67      0.90      0.77        20
         van       0.92      0.55      0.69        20

    accuracy                           0.78       160
   macro avg       0.80      0.78      0.77   